# This code is written by A. Aysu Çatal (a.aysucatal@gmail.com) to design RHEA with literature data.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

import seaborn as sns

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from matplotlib import style
from io import StringIO
from pandas import read_csv
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split
from  IPython import display
import pathlib
import shutil
import tempfile

from sklearn.preprocessing import PowerTransformer
import scipy.stats
from scipy.stats.stats import pearsonr
print(tf.__version__)

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import initializers
from keras.layers import Dense
from keras import regularizers, losses

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import linear_model

import math

In [ ]:
column_names = ['paper','Ti','Nb','Hf','Zr', 'Al','Ta', 'Mo','W','V', 
                 'Cr', 'Si', 'plas','homT', 'homt','ar', 'ardiff','Vec', 'target', 'Hmix', 'Smix'] # 'temperature','Tm',
dataframe = pd.read_csv("Hardness.csv", names=column_names, header=None, sep=';', decimal='.')
dataset = dataframe.values
np.random.shuffle(dataset)
dataframe

In [ ]:
Nb = dataframe["Nb"]
Ti =  dataframe["Ti"]
Hf = dataframe["Hf"]
Zr = dataframe["Zr"]
Al = dataframe["Al"]
Ta = dataframe["Ta"]
Mo = dataframe["Mo"]
W = dataframe["W"]
V = dataframe["V"]
Cr = dataframe["Cr"]
Si = dataframe["Si"]
plas = dataframe["plas"]
homT = dataframe["homT"]
homt = dataframe["homt"]
ar = dataframe["ar"]
ardiff = dataframe["ardiff"]
Vec = dataframe["Vec"]
paper = dataframe["paper"]
target = dataframe["target"]
#temperature = dataframe['temperature']
#Tm = dataframe["Tm"]
Hmix = dataframe["Hmix"]
Smix = dataframe["Smix"]

In [ ]:
dataframe2 =  pd.concat([Ti, Nb, Hf, Zr, Al, Ta, Mo, W, V, Cr, Si, ar, plas, homT, homt, ardiff, Vec, Hmix, Smix,  target], axis=1) # temperature, Tm,
dataset2 = dataframe2.values # Ti, Nb, Hf, Zr, Al, Ta, Mo, W, V, Cr, Si,  temperature, Tm,
df=pd.DataFrame(dataset2)
df

In [ ]:
story = {}

batch_size = 32
no_epochs = 1000

#k-fold cv
num_folds = 5
mse_per_fold = []
loss_per_fold = []
scores=[]
j=0
# K-fold Cross Validation model evaluation
kfold = KFold(n_splits=num_folds, shuffle=True)
minmse = 1000000
best=0

tf.keras.optimizers.Adam(
    learning_rate=0.0003, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)


fold_no = 1
for train_index, test_index in kfold.split(inputs):

    X_train, X_test, Y_train, Y_test = inputs[train_index],  inputs[test_index], targets[train_index], targets[test_index]
    
    pt_x=PowerTransformer(method='yeo-johnson', standardize=True)
    pt_y=PowerTransformer(method='yeo-johnson', standardize=True)
    pt_x.fit(X_train)
   
    X_train_trans=pt_x.transform(X_train)
    X_test_trans=pt_x.transform(X_test)
    pt_y.fit(Y_train.reshape(-1,1))
    Y_train_trans=pt_y.transform(Y_train.reshape(-1,1))
    Y_test_trans=pt_y.transform(Y_test.reshape(-1,1))
   

    
    node=37
    alfa=0.001
    # DEFINE THE MODEL 
    initializer = tf.keras.initializers.GlorotNormal(seed=None)
    model = Sequential([
        (Dense(node, input_dim=19, kernel_initializer=initializer, activation='relu', kernel_regularizer=regularizers.l2(alfa))),
        (Dense(node, kernel_initializer=initializer, activation='relu',kernel_regularizer=regularizers.l2(alfa))),
        #(Dense(node, kernel_initializer=initializer, activation='relu',kernel_regularizer=regularizers.l2(alfa))),
        #(Dense(node, kernel_initializer=initializer, activation='relu',kernel_regularizer=regularizers.l2(alfa))),
        (Dense(1, kernel_initializer=initializer, activation='linear'))
    ])
    

    # COMPILE
    model.compile(loss='mse',
                optimizer= 'Adam',
                metrics=['mse'])

    # FIT DATA TO THE MODEL
    history = model.fit(X_train_trans, Y_train_trans,
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=0)

    # CALCULATE THE SCORES

    scores.append(model.evaluate(X_test_trans, Y_test_trans, verbose=0))
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[j][:1]}; {model.metrics_names[1]} of {scores[j][:2]}')
    mse_per_fold.append(scores[j][:1])
    loss_per_fold.append(scores[j][:1])
    mseA=loss_per_fold.append(scores[j][:1])

    story[fold_no] = model

    if float(str(mse_per_fold[j])[1:-1]) < minmse:
        X_train_min = X_train
        X_test_min = X_test
        Y_train_min = Y_train
        Y_test_min = Y_test
        X_train_trans_min = X_train_trans
        X_test_trans_min = X_test_trans
        Y_train_trans_min = Y_train_trans
        Y_test_trans_min = Y_test_trans
        besto = fold_no
        
        minmse = float(str(mse_per_fold[j])[1:-1])


    # next fold:
    fold_no = fold_no + 1
    j = j + 1

### END OF KFOLD LOOP


# == AVERAGE SCORES ==
#print('------------------------------------------------------------------------')
#print('Score per fold')
for i in range(0, len(mse_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - MSE: {mse_per_fold[i]}')
    print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> MSE: {np.mean(mse_per_fold)} (+- {np.std(mse_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


In [ ]:
story[best].evaluate(X_test_trans_min,Y_test_trans_min)
modelbest = story[best]
test_predictions = modelbest.predict(X_test_trans_min).flatten()
train_predictions = modelbest.predict(X_train_trans_min).flatten()

In [ ]:
### PLOTS FOR KFOLD 
result_test_trans = modelbest.evaluate(X_test_trans_min, Y_test_trans_min,  verbose=0)
print(result_test_trans)

a = plt.axes(aspect='equal')
plt.scatter(Y_test_trans_min, test_predictions)
plt.xlabel('True ')
plt.ylabel('Predictions ')
lims = [-3,3]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
result_train_trans = modelbest.evaluate(X_train_trans_min, Y_train_trans_min,  verbose=0)
print(result_train_trans)

a = plt.axes(aspect='equal')
plt.scatter(Y_train_trans_min, train_predictions)
plt.xlabel('True ')
plt.ylabel('Predictions ')
import matplotlib.pyplot as plt
plt.style.use('classic')
lims = [-3,3]
plt.xlim(lims)
plt.ylim(lims)

_ = plt.plot(lims, lims)

In [ ]:
pt_y_best=PowerTransformer(method='yeo-johnson', standardize=True)
pt_y_best.fit(Y_train_min.reshape(-1,1))
train_predictions_untrans = pt_y_best.inverse_transform(train_predictions.reshape(-1,1))
test_predictions_untrans = pt_y_best.inverse_transform(test_predictions.reshape(-1,1))


In [ ]:
loss_test = mean_squared_error(test_predictions_untrans,Y_test_min)
loss_train = mean_squared_error(train_predictions_untrans, Y_train_min)
print(np.sqrt(np.mean(loss_test)))
np.sqrt(np.mean(loss_train))

In [ ]:
a = plt.axes(aspect='equal')
plt.scatter(test_predictions_untrans,Y_test_min, c='#ff7f0e')
plt.scatter(train_predictions_untrans, Y_train_min, c='#17becf')
plt.xlabel('True ')
plt.ylabel('Predictions ')
lims = [0,1000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
X_train_min = pd.DataFrame(X_train_min)
Y_train_min = pd.DataFrame(Y_train_min)
train_predictions_untrans = pd.DataFrame(train_predictions_untrans)

result_train=pd.concat([X_train_min, train_predictions_untrans, Y_train_min], axis=1)

X_test_min = pd.DataFrame(X_test_min)
Y_test_min = pd.DataFrame(Y_test_min)
test_predictions_untrans = pd.DataFrame(test_predictions_untrans)

result_test=pd.concat([X_test_min, test_predictions_untrans, Y_test_min], axis=1)


results = pd.concat([result_test, result_train], axis=0)


In [ ]:
X_test.shape

In [ ]:
results.to_csv (r'C:\Users\acatal19\Desktop\hea\data setler upgraded\results_NN\eniyiuc_data.csv', index = False, header=True,sep=',', decimal='.')
